In [37]:
! python -m spacy download en_vectors_web_lg

    100% |████████████████████████████████| 661.8MB 23.7MB/s ta 0:00:01   10% |███▏                            | 66.6MB 34.2MB/s eta 0:00:18    18% |█████▉                          | 121.4MB 22.3MB/s eta 0:00:25    23% |███████▌                        | 155.4MB 24.6MB/s eta 0:00:21    23% |███████▋                        | 156.7MB 23.3MB/s eta 0:00:22    31% |██████████                      | 207.6MB 23.4MB/s eta 0:00:20    32% |██████████▌                     | 218.1MB 21.2MB/s eta 0:00:21    48% |███████████████▋                | 322.7MB 21.1MB/s eta 0:00:17    59% |███████████████████             | 394.6MB 32.1MB/s eta 0:00:09    59% |███████████████████▏            | 395.9MB 23.3MB/s eta 0:00:12    61% |███████████████████▊            | 407.7MB 21.7MB/s eta 0:00:12    64% |████████████████████▊           | 428.6MB 22.4MB/s eta 0:00:11    67% |█████████████████████▊          | 448.2MB 23.2MB/s eta 0:00:10    74% |████████████████████████        | 495.3MB 20.6MB/s eta 0:00:09    76% 

In [45]:
import spacy
from spacy.tokens.token import Token
from numpy import dot
from numpy.linalg import norm

In [39]:
nlp = spacy.load('en_vectors_web_lg')

[ 5.14087021e-01 -2.78459996e-01  2.42767006e-01  4.54899669e-02
 -2.59425014e-01 -3.19999963e-01  3.23920012e-01 -6.71030045e-01
 -9.98499990e-02  1.91499996e+00 -5.68080008e-01 -2.74451017e-01
 -1.49906695e-01  8.01083148e-02 -2.34764010e-01 -1.10950008e-01
 -1.02593988e-01  8.53819966e-01 -2.68564999e-01  3.85140002e-01
 -1.36149988e-01  6.35029972e-01 -7.62044966e-01 -2.52770007e-01
 -6.75969958e-01  3.89851004e-01 -2.89680034e-01  1.75860003e-01
 -5.16229987e-01  5.21373034e-01 -1.89909995e-01  6.73759937e-01
  1.17550008e-01 -4.69896019e-01  5.88999987e-01  1.29447982e-01
 -5.71900010e-01 -5.47450066e-01 -4.84210014e-01  5.85503951e-02
  4.82379973e-01 -2.86769986e-01 -2.01718003e-01 -4.74729985e-01
  3.43068987e-01 -2.28827983e-01 -1.76439017e-01  6.05450034e-01
  2.07139999e-01 -2.89762974e-01 -7.63288975e-01  4.37090009e-01
 -2.06220001e-01 -4.20252979e-01  1.98040009e-01  3.18709970e-01
 -9.51815993e-02 -3.23054016e-01 -6.02343976e-01  2.33427018e-01
 -2.15409994e-02 -6.29774

In [48]:
vecs = nlp('king man woman')
out_vec = vecs[0].vector - vecs[1].vector + vecs[2].vector
print(out_vec)

[ 5.14087021e-01 -2.78459996e-01  2.42767006e-01  4.54899669e-02
 -2.59425014e-01 -3.19999963e-01  3.23920012e-01 -6.71030045e-01
 -9.98499990e-02  1.91499996e+00 -5.68080008e-01 -2.74451017e-01
 -1.49906695e-01  8.01083148e-02 -2.34764010e-01 -1.10950008e-01
 -1.02593988e-01  8.53819966e-01 -2.68564999e-01  3.85140002e-01
 -1.36149988e-01  6.35029972e-01 -7.62044966e-01 -2.52770007e-01
 -6.75969958e-01  3.89851004e-01 -2.89680034e-01  1.75860003e-01
 -5.16229987e-01  5.21373034e-01 -1.89909995e-01  6.73759937e-01
  1.17550008e-01 -4.69896019e-01  5.88999987e-01  1.29447982e-01
 -5.71900010e-01 -5.47450066e-01 -4.84210014e-01  5.85503951e-02
  4.82379973e-01 -2.86769986e-01 -2.01718003e-01 -4.74729985e-01
  3.43068987e-01 -2.28827983e-01 -1.76439017e-01  6.05450034e-01
  2.07139999e-01 -2.89762974e-01 -7.63288975e-01  4.37090009e-01
 -2.06220001e-01 -4.20252979e-01  1.98040009e-01  3.18709970e-01
 -9.51815993e-02 -3.23054016e-01 -6.02343976e-01  2.33427018e-01
 -2.15409994e-02 -6.29774

In [49]:

# cosine similarity
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

def get_most_similar_word(vec, model):
    print(len(model.vocab))
    # gather all known words, take only the lowercased versions
    allWords = list({w for w in model.vocab if w.has_vector})
    
    allWords.sort(key=lambda w: cosine(w.vector, vec))
    allWords.reverse()
    print("Top 10 most similar words:")
    for word in allWords[:10]:
        print(word.orth_)

In [50]:
get_most_similar_word(out_vec, nlp)

1071393


/home/jendrik/.local/share/virtualenvs/dlworkshop-VuPJaqFs/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in float_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


Top 10 most similar words:
fornaro
Post-Teledyne
deferring
THERMAL-PROCESS
LLADRO
sketchers
Otakar
Busacker
Erlandson
kaczorowski
